In [1]:
import numpy as np
import joblib  # for saving the model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import KernelPCA, PCA
from sklearn.neighbors import KNeighborsClassifier
import optuna
import optuna.visualization as vis

c:\Users\berma\Documents\Mathematics for Machine Learning\MML-CW\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
n = 9

X = np.load('../../Datasets_Train_Test_Split/kryptonite-%s-X_train.npy' % n)
y = np.load('../../Datasets_Train_Test_Split/kryptonite-%s-y_train.npy' % n)

X_train_total, X_test, y_train_total, y_test = train_test_split(X, y, test_size=0.1, random_state=42)  # 10% of all data for final model evaluation

# To hold the best model and accuracy score
best_model = None
best_accuracy = -1
SAVE_BEST_MODEL = False

In [7]:
def objective(trial):
    global best_model, best_scaler, best_accuracy, best_pca

    # Shuffle and split data
    X_train, X_val, y_train, y_val = train_test_split(X_train_total, y_train_total, test_size=0.8, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    pca = PCA(n_components=trial.suggest_int('n_components', 1, X_train_scaled.shape[1]))
    X_train_scaled = pca.fit_transform(X_train_scaled)
    X_val_scaled = pca.transform(X_val_scaled)

    # KNN Classifier parameters
    classifier = KNeighborsClassifier(
        n_neighbors=trial.suggest_int('n_neighbors', 1, 50),
        weights=trial.suggest_categorical('weights', ['uniform', 'distance']),
        n_jobs=-1
    )

    # Train the classifier
    classifier.fit(X_train_scaled, y_train)

    # Evaluate on the validation set
    y_val_pred = classifier.predict(X_val_scaled)
    val_accuracy = accuracy_score(y_val, y_val_pred)

    # Report validation accuracy for pruning
    trial.report(val_accuracy, step=0)

    # Check if the trial should be pruned
    if trial.should_prune():
        raise optuna.TrialPruned()

    # If this is the best model so far, save it
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_model = classifier  # Update the best model
        best_scaler = scaler
        best_pca = pca

        if SAVE_BEST_MODEL:
            # Save the best model
            joblib.dump(best_model, 'best_knn_model.joblib')
            print(f"New best model saved with accuracy: {best_accuracy:.4f}")

    return val_accuracy

In [8]:
# Create a study object with a pruner
pruner = optuna.pruners.MedianPruner(n_startup_trials=10)
study = optuna.create_study(direction="maximize", pruner=pruner)

# Optimize the objective function
study.optimize(objective, n_trials=100)

print("Best hyperparameters:", study.best_params)
print("Best value:", study.best_value)

# Evaluate on the validation set
X_test_scaled = best_scaler.transform(X_test)
X_test_scaled = best_pca.transform(X_test_scaled)
y_test_pred = best_model.predict(X_test_scaled)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f'accuracy on test set:{test_accuracy}')

# Optional: Visualize optimization results
vis.plot_optimization_history(study).show()
vis.plot_param_importances(study).show()
vis.plot_slice(study).show()

[I 2024-11-25 17:22:13,828] A new study created in memory with name: no-name-be0f4f11-7958-41ff-8b17-3034a5f1bc96
[I 2024-11-25 17:22:14,215] Trial 0 finished with value: 0.3160686728395062 and parameters: {'n_components': 9, 'n_neighbors': 37, 'weights': 'uniform'}. Best is trial 0 with value: 0.3160686728395062.
[I 2024-11-25 17:22:14,493] Trial 1 finished with value: 0.7199074074074074 and parameters: {'n_components': 6, 'n_neighbors': 11, 'weights': 'uniform'}. Best is trial 1 with value: 0.7199074074074074.
[I 2024-11-25 17:22:14,712] Trial 2 finished with value: 0.4923804012345679 and parameters: {'n_components': 2, 'n_neighbors': 26, 'weights': 'uniform'}. Best is trial 1 with value: 0.7199074074074074.
[I 2024-11-25 17:22:15,102] Trial 3 finished with value: 0.3150077160493827 and parameters: {'n_components': 9, 'n_neighbors': 36, 'weights': 'uniform'}. Best is trial 1 with value: 0.7199074074074074.
[I 2024-11-25 17:22:15,203] Trial 4 finished with value: 0.7515432098765432 an

Best hyperparameters: {'n_components': 8, 'n_neighbors': 3, 'weights': 'distance'}
Best value: 0.9475308641975309
accuracy on test set:0.9506944444444444
